# Fine Foods - Factorization Machine Analysis
We will be using food reviews from Amazon to build a recommendation engine using Factorization Machine in SAS VIYA

Factorization Machine (FM) is one of the newest algorithms in the Machine Learning space, and has been developed in SAS. FM is a general prediction algorithm, similar to Support Vector Machines, that can model very sparce data, an area where traditional Machine Learning techniques fail. 

Since FM is a general prediction algorithm, it can accept any sized real vector as inputs. Because of this, we will use SAS Viya text analytics capabilities to represent text as numeric vectors, that we can use as inputs to our FM model.


this notebook has **three** parts:
1. Load Data &  Prepare Notebook
2. Data overview & manipulation
3. View & Promote finalized Dataests
4. Perform Analyses
5. Deploy model in Event Stream Processing

We will use the dataset promoted to public memory to train our FM model in SAS Studio

## 1. Load Data
In this step, we will make a connection to our CAS server, and will load the revelant table that we prepared in Python into memory


In [13]:
from swat import *
import _config
%matplotlib inline
#swat.options.cas.print_messages = False

# Connect to the session
cashost='gtpviyaea12.unx.sas.com'
casport=5570
user,pswd = _config.login()

s = CAS(cashost, casport, user, pswd)

# Create a CAS library called DMLib pointing to the defined directory
## Note, need to specify the srctype is path, otherwise it defaults to HDFS
s.table.addCaslib(datasource={'srctype':'path'}, name='DMlib', path="/viyafiles/ankram/Data");

#Load Data
f = 'foods_prepped1'
s.loadTable(caslib='DMlib', path=f +'.csv', casout=f);

#Load actionsets
actionsets=['fedSQL', 'autoTune', 'factMac', 'textMining']
[s.builtins.loadactionset(i) for i in actionsets];

#Create shortcuts
food = s.CASTable(f)
target = 'score'
#class_inputs = ['helpfulness','productid','time','userid']

NOTE: 'DMlib' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 'DMlib'.
NOTE: Cloud Analytic Services made the file foods_prepped1.csv available as table FOODS_PREPPED1 in caslib DMlib.
NOTE: Added action set 'fedSQL'.
NOTE: Added action set 'autoTune'.
NOTE: Added action set 'factMac'.
NOTE: Added action set 'textMining'.


## 2. Overview Data & Manipulation
In this step, we will take a look at the data presented to us. We notice that this dataset contains the variables "productId", "userid", and "score". This is the most basic information required to build a recommender system through traditional collaborative filtering methodologies.

While traditional collaborative filtering methods are powerful in their prediction abilities, they are severely restricted by the input vector requirements, typically taking only two inputs - user and item. With factorization machine, we can enrich our models by having a much wider input vector containing more than two inputs. We can see if "Time" and "helpfulness" can make our engine more accurate.

Lastly, each user has some amount of free form text. By using text mining capabilities in Viya, we can summarize the text into topics, which we can then include in our factorization machine model.

Note - We can quickly view the data with the 'head' method 

#Add a column Identifier in-memory
s.dataStep.runCode('''
    data ''' + f + '''; 
        set '''  + f + ''';
        if key<10000;
    run;,
    
    replace="YES"
''')


In [14]:
#Print Number of reviews
print(len(food), "Reviews")

#Validate first few rows
food.head(5)

568454 Reviews


,key,helpfulness,productId,score,summary,text,time,userId
0,296083.0,2/2,B0001VWG5A,5.0,Just what my food needed!!!!,I am so happy I found this product!!! In Latin...,1.330906e+09,A26E6STZIFGT9Q
1,296084.0,3/3,B000VQ9I92,4.0,Better with a simple addition,These are the classic Spicy Korean noodle bowl...,1.238026e+09,A35B4VHHGZT2TP
2,296085.0,0/0,B000VQ9I92,5.0,Taste great,These noodles are great I was stationed in Kuw...,1.319501e+09,A1IOQ3A4DY1B52
3,296086.0,1/4,B000VQ9I92,3.0,yummy,I do love this brand of soup very much but be ...,1.197331e+09,A2SBVXOOCT18KF
4,296087.0,0/0,B002ONOCBY,4.0,Perfect for Dunking!,Rich Tea biscuits are really not the most exci...,1.316304e+09,A1VN7PA3TYI8RO


### Fix Date Field
We will create two character variables to use:
1. Weekday: 1(Sunday) - 7(Saturday)
2. Month: 1(January) - 12(December)

In [15]:
s.dataStep.runCode('''
    data ''' + f + '''; 
        set '''  + f + ''';
            
            datetm=time + "01jan1970 0:0:0"dt ;
                Format datetm datetime. ;
                   
            date=datepart(datetm);
                format date mmddyy10.;

            weekday=weekday(date);
            
            month=substr(put(date, mmddyy10.),1,2);
            
    run;,
    replace="YES"
''')

food.head()

,key,helpfulness,productId,score,summary,text,time,userId,datetm,date,weekday,month
0,375803.0,1/2,B000HBBJ3O,1.0,Bad tea,There is virtually no tea in the tea bags. Sav...,1.342570e+09,A2UGV0RCFJW3P8,1.658189e+09,19192.0,4.0,07
1,375804.0,4/4,B003CJN1CA,5.0,GREAT COFFEE,This is a great decaf coffee which rivals (but...,1.310861e+09,AJWN139LPB56L,1.626480e+09,18825.0,1.0,07
2,375805.0,4/4,B003CJN1CA,4.0,"lovely flavor, too finely ground",I love Douwe Egberts coffee - it has a great f...,1.298246e+09,A1RXIBJ2DDIRPM,1.613866e+09,18679.0,2.0,02
3,375806.0,4/4,B003CJN1CA,5.0,Great Aroma for Decaf Coffee,This coffee has an incredible aroma for a deca...,1.297555e+09,A26D39BQZS0A1Y,1.613174e+09,18671.0,1.0,02
4,375807.0,2/2,B003CJN1CA,5.0,very tasty,i first tasted this coffe at a casino and was ...,1.300147e+09,A12XMSVYFI71GJ,1.615766e+09,18701.0,3.0,03


## 4. View Finalized Dataset for analysis

Our data prep is now complete. After doing some data preparation and some text analytics, we now have the following columns to use in our recommendation engine analysis:
1. helpfulness
2. productId
3. weekday
4. day_month,
5. userId
6. _TextTopic_1 - _TextTopic_5
7. Score (Target variable)

Once we are satisfied with our dataset, we can promote the dataset into public memory (making it avaiable to other interfaces and users) or we can save the data to disc.

In [16]:
s.CASTable(f).head()

,key,helpfulness,productId,score,summary,text,time,userId,datetm,date,weekday,month
0,375803.0,1/2,B000HBBJ3O,1.0,Bad tea,There is virtually no tea in the tea bags. Sav...,1.342570e+09,A2UGV0RCFJW3P8,1.658189e+09,19192.0,4.0,07
1,375804.0,4/4,B003CJN1CA,5.0,GREAT COFFEE,This is a great decaf coffee which rivals (but...,1.310861e+09,AJWN139LPB56L,1.626480e+09,18825.0,1.0,07
2,375805.0,4/4,B003CJN1CA,4.0,"lovely flavor, too finely ground",I love Douwe Egberts coffee - it has a great f...,1.298246e+09,A1RXIBJ2DDIRPM,1.613866e+09,18679.0,2.0,02
3,375806.0,4/4,B003CJN1CA,5.0,Great Aroma for Decaf Coffee,This coffee has an incredible aroma for a deca...,1.297555e+09,A26D39BQZS0A1Y,1.613174e+09,18671.0,1.0,02
4,375807.0,2/2,B003CJN1CA,5.0,very tasty,i first tasted this coffe at a casino and was ...,1.300147e+09,A12XMSVYFI71GJ,1.615766e+09,18701.0,3.0,03


In [17]:
#Load data into Public memory
#s.table.promote(table='docpro')

#Save file to Server
s.table.save(caslib='DMlib', name='Foods_prep_text_GTP_stream.sashdat', table=f, replace=True)

##Load data into Public memory
#s.loadTable(caslib='DMlib',  path='Foods_prep_text_GTP'+'.sashdat', casout='Foods_Prep_Text')

NOTE: Cloud Analytic Services saved the file Foods_prep_text_GTP_stream.sashdat in caslib DMlib.


[caslib]

 'DMlib'

[name]

 'Foods_prep_text_GTP_stream.sashdat'

+ Elapsed: 3.79s, user: 0.17s, sys: 0.737s, mem: 186mb

In [2]:
s.loadTable(caslib='DMlib',  path='Foods_prep_text_GTP_stream'+'.sashdat', casout='Foods_Prep_Text1')
s.CASTable('Foods_Prep_Text1').head()

NOTE: Cloud Analytic Services made the file Foods_prep_text_GTP.sashdat available as table FOODS_PREP_TEXT1 in caslib DMlib.


,helpfulness,productId,score,summary,text,time,userId,weekday,day_month,key,_Col1_,_Col2_,_Col3_,_Col4_,_Col5_,_TextTopic_1,_TextTopic_2,_TextTopic_3,_TextTopic_4,_TextTopic_5
0,0/0,B004QDA8WC,2.0,WAY too SWEET for me!,As someone who really enjoys a Chai Tea Latte ...,1.350605e+09,A19SQLKMY7PEEI,6.0,10/19,463584.0,0.841102,0.139877,0.059419,0.495123,0.155901,1.0,0.0,0.0,0.0,0.0
1,0/0,B004QDA8WC,3.0,Great change from coffee,Got this as a present for my son for our Keuri...,1.350605e+09,A10O4LYO967IZ,6.0,10/19,463585.0,0.719527,0.372908,0.051278,0.573038,0.110539,0.0,0.0,0.0,0.0,0.0
2,0/0,B004QDA8WC,3.0,Not a bad chai latte but expensive,If I could I'd give it 3.5 stars.<br /><br />T...,1.350259e+09,A1PATZ96CQ5X7K,2.0,10/15,463586.0,0.690319,0.092326,0.154206,0.665715,0.219043,0.0,0.0,0.0,0.0,0.0
3,0/0,B004QDA8WC,5.0,Comfort in a cup,"This Chai Latte is a ""go to"" when you want a l...",1.350173e+09,ALEABNMSVO1JI,1.0,10/14,463587.0,0.615672,0.269603,0.072997,0.664087,0.319254,0.0,0.0,0.0,0.0,0.0
4,0/0,B004QDA8WC,1.0,Not a fan,I'm particular when it comes to chai tea but I...,1.350000e+09,A25L6BAHYZH652,6.0,10/12,463588.0,0.973065,-0.105761,0.019914,0.088694,0.183566,1.0,0.0,0.0,0.0,0.0


## V. Perform Anaysis

We will perform four different FM models to predict score:
1. score = userID + productId
2. score = userID + productId + weekday 
3. score = userID + productId + month
4. score = userID + productId + weekday + month

### Model 1
In this first model, we will use 'userId' and 'productId' to predict the score (1-5) that a user will give to a restaurant. 
<br>

the algorithm converges and returns a rmse of 1.04. This means that we were on average, holding everything else constatant, about one star off

In [3]:
s.loadactionset('autotune')

class_inputs = ['userId', 'productId']

training_options = dict(
                    table     = 'Foods_Prep_Text1',
                    inputs    = class_inputs,
                    nominals  = class_inputs,
                    target    = target,
                    seed      = 123,
                    savestate = dict(name = 'fm_model_short', replace = True))

s.invoke('autotune.tuneFactMac', trainOptions=training_options,
                                           tunerOptions=dict(maxTime=300, validationPartitionFraction=0.1),
                                           tuningParameters=[dict(namePath='nfactors', initValue=2)])
for response in s:
    for k, v in response:
        print(k, v)

NOTE: Added action set 'autotune'.
NOTE: Autotune is started for 'Factorization Machine' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXEVALS=50.
NOTE: Autotune option MAXTIME=300 (sec.).
NOTE: Autotune objective is 'Root Average Squared Error'.
NOTE: Autotune number of parallel evaluations is set to 1, each using 3 worker nodes.
NOTE: The INITVALUE '2' of tuning parameter NFACTORS is not included in the VALUELIST for this parameter.
NOTE: The initial point will not be used to seed the tuning process.
NOTE: The seed for random partition/fold generation is 123.
         Iteration       Evals     Best Objective        Time
                 0           1             1.1066        7.52
                 1           8             1.0381      302.38
NOTE: Autotune process reached maximum tuning time.
NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization ma

### Model 2
In this second model, we will use 'userId', 'productId', 'helpfulness','weekday',and 'day_month to predict the score (1-5) that a user will give to a restaurant. 
<br>

The 'time' variable is just a UNIX timestamp representing the number of seconds since Junuary, 1971. Only days were recorded, no hours or seconds.  However, training the model on a fixed time makes it hard to extrapolate into the future. Because of this, two variables were derived  from time:
1. weekday - Sunday (1) to monday (7)
2. day_month - day and month of timestamp. Example 02May
<br>
the algorithm converges and returns a rmse of .99. This means that we were on average, holding everything else constatant, about one star off

In [9]:
s.loadactionset('autotune')

class_inputs = ['userId', 'productId','weekday']
learnstep = [.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

training_options = dict(
                    table     = f,
                    inputs    = class_inputs,
                    nominals  = class_inputs,
                    target    = target,
                    seed      = 123,
                    savestate = dict(name = 'fm_model_short', replace = True))

s.invoke('autotune.tuneFactMac', trainOptions=training_options,
                                           tunerOptions=dict(maxTime=500, validationPartitionFraction=0.1),
                                           tuningParameters=[dict(namePath='learnstep', valuelist=learnstep)])

for response in s:
    for k, v in response:
        print(k, v)

NOTE: Added action set 'autotune'.
NOTE: Autotune is started for 'Factorization Machine' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXEVALS=50.
NOTE: Autotune option MAXTIME=500 (sec.).
NOTE: Autotune objective is 'Root Average Squared Error'.
NOTE: Autotune number of parallel evaluations is set to 1, each using 3 worker nodes.
NOTE: The INITVALUE '0.001' of tuning parameter LEARNSTEP is not included in the VALUELIST for this parameter.
NOTE: The initial point will not be used to seed the tuning process.
NOTE: The seed for random partition/fold generation is 123.
         Iteration       Evals     Best Objective        Time
                 0           1             1.2339       10.30
NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
NOTE: >>> Progress: completed iteration 1
NOTE: >>> Progress: completed iteration 2
NOTE: >>> 

ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.
ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.
ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.
ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.


NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
NOTE: >>> Progress: completed iteration 1
NOTE: >>> Progress: completed iteration 2
NOTE: >>> Progress: completed iteration 3
NOTE: >>> Progress: completed iteration 4
NOTE: >>> Progress: completed iteration 5
NOTE: >>> Progress: completed iteration 6
NOTE: >>> Progress: completed iteration 7
NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
NOTE: >>> Progress: completed iteration 1
NOTE: >>> Progress: completed iteration 2
NOTE: >>> Progress: completed iteration 3
NOTE: >>> Progress: completed iteration 4
NOTE: >>> Progress: completed iteration 5
NOTE: >>> Progress: completed iteration 6
NOTE: >>> Progress: completed iteration 7
NOTE: >>> Progress: completed iteration 8
NO

FitStat Fit Statistics for _AUTOTUNE_DEFAULT_SCORE_TABLE_

      NOBS       ASE      DIV      RASE       MAE      RMAE      MSLE  \
0  37106.0  1.063823  37106.0  1.031418  0.628333  0.792674  0.073044   

      RMSLE  
0  0.270267  
TunerInfo Tuner Information

                         Parameter                       Value
0                       Model Type       Factorization Machine
1         Tuner Objective Function  Root Average Squared Error
2                    Search Method                          GA
3              Maximum Evaluations                          50
4                  Population Size                          10
5               Maximum Iterations                           5
6   Maximum Tuning Time in Seconds                         500
7                  Validation Type            Single Partition
8    Validation Partition Fraction                        0.10
9                        Log Level                           2
10                            Seed          

In [19]:
s.loadactionset('autotune')

class_inputs = ['userId', 'productId','month']
learnstep = [.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

training_options = dict(
                    table     =  f,
                    inputs    = class_inputs,
                    nominals  = class_inputs,
                    target    = target,
                    seed      = 123,
                    savestate = dict(name = 'fm_model_short', replace = True))

s.invoke('autotune.tuneFactMac', trainOptions=training_options,
                                           tunerOptions=dict(maxTime=500, validationPartitionFraction=0.1),
                                           tuningParameters=[dict(namePath='learnstep', valuelist=learnstep)])

for response in s:
    for k, v in response:
        print(k, v)

NOTE: Added action set 'autotune'.
NOTE: Autotune is started for 'Factorization Machine' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXEVALS=50.
NOTE: Autotune option MAXTIME=500 (sec.).
NOTE: Autotune objective is 'Root Average Squared Error'.
NOTE: Autotune number of parallel evaluations is set to 1, each using 3 worker nodes.
NOTE: The INITVALUE '0.001' of tuning parameter LEARNSTEP is not included in the VALUELIST for this parameter.
NOTE: The initial point will not be used to seed the tuning process.
NOTE: The seed for random partition/fold generation is 123.
         Iteration       Evals     Best Objective        Time
                 0           1             1.2222        9.78
NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
NOTE: >>> Progress: completed iteration 1
NOTE: >>> Progress: completed iteration 2


ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.
ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.
ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.


NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
NOTE: >>> Progress: completed iteration 1
NOTE: >>> Progress: completed iteration 2
NOTE: >>> Progress: completed iteration 3
                 1          11             1.0205      375.68
NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
NOTE: >>> Progress: completed iteration 1
NOTE: >>> Progress: completed iteration 2
NOTE: >>> Progress: completed iteration 3
NOTE: >>> Progress: completed iteration 4
NOTE: >>> Progress: completed iteration 5
NOTE: >>> Progress: completed iteration 6
NOTE: >>> Progress: completed iteration 7
NOTE: >>> Progress: completed iteration 8
NOTE: >>> Progress: completed iteration 9
NOTE: >>> Progress: completed iteration 10
NOTE: >>> Progress: com

ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.


    Progress  Objective       Loss
0        1.0  21.810322  21.810322
1        2.0  13.321495  13.321495
2        3.0   9.840958   9.840958
3        4.0   7.630255   7.630255
4        5.0   5.906391   5.906391
5        6.0   4.599838   4.599838
6        7.0   3.487571   3.487571
7        8.0   2.716327   2.716327
8        9.0   1.898818   1.898818
9       10.0   1.414475   1.414475
10      11.0   1.112225   1.112225
11      12.0   0.914503   0.914503
12      13.0   0.774362   0.774362
13      14.0   0.677039   0.677039
14      15.0   0.582300   0.582300
15      16.0   0.510280   0.510280
16      17.0   0.467491   0.467491
17      18.0   0.411494   0.411494
18      19.0   0.374206   0.374206
19      20.0   0.339312   0.339312
20      21.0   0.310522   0.310522
21      22.0   0.283314   0.283314
22      23.0   0.263633   0.263633
23      24.0   0.244698   0.244698
24      25.0   0.227226   0.227226
25      26.0   0.214788   0.214788
26      27.0   0.198588   0.198588
27      28.0   0.194

### Model 3
In this second model, we will use 'userId', 'productId', 'helpfulness','weekday', 'day_month', and five text variables (_Col1_-_Col5_) to predict the score (1-5) that a user will give to a restaurant. 
<br>

Using the Text Analytics process described above, we created five variables, that we can  call "topics", that summarize the text across various latent dimensions. We will try to see if these five topics can make our prediction more accurate.

the algorithm converges and returns a rmse of 3.32. This means that the text cataegories are to generic, and fail to help with prediction. The addition of these five new variables overfit our model, and made the model worse when scoring the validation.

In [21]:
s.loadactionset('autotune')

class_inputs = ['userId', 'productId','weekday','month']

learnstep = [.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,.99]

training_options = dict(
                    table     = f,
                    inputs    = class_inputs,
                    nominals  = class_inputs,
                    target    = target,
                    seed      = 123,
                    savestate = dict(name = 'fm_model_short', replace = True))

s.invoke('autotune.tuneFactMac', trainOptions=training_options,
                                           tunerOptions=dict(maxTime=500, validationPartitionFraction=0.1),
                                           tuningParameters=[dict(namePath='nfactors', initValue=2)])

for response in s:
    for k, v in response:
        print(k, v)

NOTE: Added action set 'autotune'.
NOTE: Autotune is started for 'Factorization Machine' model.
NOTE: Autotune option SEARCHMETHOD='GA'.
NOTE: Autotune option MAXEVALS=50.
NOTE: Autotune option MAXTIME=500 (sec.).
NOTE: Autotune objective is 'Root Average Squared Error'.
NOTE: Autotune number of parallel evaluations is set to 1, each using 3 worker nodes.
NOTE: The INITVALUE '2' of tuning parameter NFACTORS is not included in the VALUELIST for this parameter.
NOTE: The initial point will not be used to seed the tuning process.
NOTE: The seed for random partition/fold generation is 123.
         Iteration       Evals     Best Objective        Time
                 0           1             1.1018        9.08
NOTE: Using SEED=123.
NOTE: Beginning data reading and levelization...
NOTE: Data reading and levelization complete.
NOTE: Beginning optimization of the factorization machine model...
                 1          10                  0      559.76
NOTE: Autotune process reached maximu

ERROR: The optimization has encountered numerical problems. Consider reducing the LEARNSTEP= parameter and/or standardizing the target variable.
ERROR: The optimization did not converge.
ERROR: The action stopped due to errors.


FitStat Fit Statistics for _AUTOTUNE_DEFAULT_SCORE_TABLE_

      NOBS       ASE      DIV      RASE       MAE      RMAE      MSLE    RMSLE
0  36867.0  1.062388  36867.0  1.030722  0.580774  0.762085  0.073089  0.27035
TunerInfo Tuner Information

                         Parameter                       Value
0                       Model Type       Factorization Machine
1         Tuner Objective Function  Root Average Squared Error
2                    Search Method                          GA
3              Maximum Evaluations                          50
4                  Population Size                          10
5               Maximum Iterations                           5
6   Maximum Tuning Time in Seconds                         500
7                  Validation Type            Single Partition
8    Validation Partition Fraction                        0.10
9                        Log Level                           2
10                            Seed                         12

## VI:  Deploy Model in Event Stream Processing
Model 2 was our most accurate model, with a rmse of .98, using 'userId', 'productId', 'helpfulness','weekday',and 'day_month to predict the score (1-5) that a user will give to a restaurant
<br>

We will now deploy the model in real-time, making five recommendations for each user as new reviews come in. We will deploy the  model with the following the steps:
1. Save the model as an Analytics Store
2. Register the model with Event Stream Processing
3. Stream data and make 

In [5]:
s.table.save(table='fm_model_short', name='factmac_astore_esp_blog', caslib='DMlib', replace=True)

NOTE: Cloud Analytic Services saved the file factmac_astore_esp_blog.sashdat in caslib DMlib.


[caslib]

 'DMlib'

[name]

 'factmac_astore_esp_blog.sashdat'

+ Elapsed: 1.16s, user: 0.02s, sys: 0.11s, mem: 245mb

In [12]:
s.close()